## Omnibus Project
### Population Current State Analysis

### Summary:
Population: Traders (incoporated sellers) active shop status, and active user state, with active listings(?), who reside in EU or ship to EU.<br>

Analysis Steps:
 - Current return profile
 - Traders currently providing contact details on shop profile

In [1]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/bigquery'])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=g3y8LkzlsandE9mvb8XwFqz9sFvj4A&access_type=offline


Unable to create credentials directory.


In [2]:
import pandas as pd
import numpy as np
from tableone import TableOne

In [3]:
# Instantiate the BigQuery client (using project ID and credentials)
from google.cloud import bigquery
project_id = "etsy-bigquery-adhoc-prod"
client = bigquery.Client(project=project_id, credentials=credentials)

### Current return profile
### Structured return policy on active traders

In [4]:
query2 = """


with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- active incorporated sellers who has a shipping profile (can ship to EU)
active_traders as (select distinct sb.shop_id,
sb.user_id,
sb.seller_tier_new,
sb.country_name, -- where the shop is based at
case when sb.country_name in (select name from eu_countries) then 'EU Seller' 
     else 'Other Seller' end as seller_country
-- l.is_active,
-- sb.user_state,
-- sb.shop_status,
-- s.trader_distinction
from `etsy-data-warehouse-prod`.rollups.seller_basics sb
inner join `etsy-data-warehouse-prod.etsy_payments.shop_business_verification_attempts` kyb  -- filter on sellers who went through KYB to identify traders                                                
on sb.shop_id = kyb.shop_id    
--left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s --new table to identify traders
--on sb.shop_id = s.shop_id
inner join `etsy-data-warehouse-prod.etsy_shard.shipping_profile` s       -- inner join to filter out shops with no shipping profile? –Laws are applicable to traders who ships, filtering out people who sell digital product, we don’t need to display message to them.
on sb.shop_id = s.shop_id 
where sb.user_state = 'active' -- active users only
and sb.shop_status = 'active' -- active shops only
and s.is_deleted = 0
-- and s.trader_distinction = 1  -- incorporated sellers only
-- and l.shop_id in (5928292, 5447890, 15903859, 20456727, 23763260, 29784414)
),
-- select count(distinct user_id), count(*) from active_traders --110912, 110912


-- get shipping profile shops (all active traders with active listings should be included in this table)
shipping_profile as (select distinct a.shop_id,
a.user_id,
-- a.country_name as seller_base_country, 
-- sp.origin_country_id, -- shipping origin country id
-- countries.name as shipping_country,
-- sp.ships_everywhere,
min(case when a.country_name in (select name from eu_countries) and ships_everywhere = 0 then '2-EU Origin Ship Country/Ships Locally' 
      when ships_everywhere =1 then '1-EU Origin/Ships Everywhere' else '3-Doesnt Ship Everywhere' end) as ship_profile
from active_traders a
left join `etsy-data-warehouse-prod.etsy_shard.shipping_profile`  sp
on a.shop_id = sp.shop_id
left join `etsy-data-warehouse-prod.etsy_v2.countries` countries
on sp.origin_country_id = countries.country_id
where sp.is_deleted = 0 -- active shipping profile only
-- and a.shop_id in (23077606, 34769184, 22176230, 31070274, 29949827)
group by a.shop_id, a.user_id),

return_policy as (
select distinct
l.shop_id, 
p.return_items_within_days  -- to get how many days allowed for return to define structured policy, structured policy means store has defined return policy
from `etsy-data-warehouse-prod`.listing_mart.listings l
inner join `etsy-data-warehouse-prod`.rollups.seller_basics b
on l.shop_id = b.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s  -- middle table to connect shop_id to structured_policies_id
on l.shop_id = s.shop_id
left join `etsy-data-warehouse-prod`.etsy_shard.shop_structured_policies p  --one shop can only have one structured return policy
on s.structured_policies_id = p.structured_policies_id
where l.is_active = 1  -- why filter on listing is active? We only want to target shops with active listings that are traders who went through KYB
and b.active_seller_status = 1  
and p.return_items_within_days is not null  --why?
and return_policy_type = "structured"  --why?
)


-- get structured return policy on these active traders
select a.user_id,
a.seller_tier_new,
a.seller_country,
sp.ship_profile,
r.return_items_within_days
from active_traders a
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s 
on a.shop_id = s.shop_id
left join shipping_profile sp
on a.shop_id = sp.shop_id
left join return_policy r
on s.shop_id = r.shop_id
"""

In [5]:
query_job = client.query(query2)
structured_return_profile = pd.read_gbq(query=query2, project_id=project_id, 
#                                         progress_bar_type='tqdm'
                                    credentials=credentials)

In [6]:
structured_return_profile.shape

(172837, 5)

In [7]:
structured_return_profile.head(10)

,user_id,seller_tier_new,seller_country,ship_profile,return_items_within_days
0,550246266,Closed Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
1,116806876,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,7
2,449917069,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
3,485091231,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,30
4,147997037,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
5,294770144,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
6,290148584,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,30
7,32413570,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
8,557612642,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>
9,619506350,Top Shop,Other Seller,1-EU Origin/Ships Everywhere,<NA>


In [8]:
structured_return_profile['seller_country'] = 'Non-Omnibus affected seller'
structured_return_profile.loc[structured_return_profile['ship_profile']!='3-Doesnt Ship Everywhere', 'seller_country'] = 'EU seller/Omnibus affected'
pd.crosstab(structured_return_profile['ship_profile'], structured_return_profile['seller_country'])

seller_country,EU seller/Omnibus affected,Non-Omnibus affected seller
ship_profile,,
1-EU Origin/Ships Everywhere,137190,0
2-EU Origin Ship Country/Ships Locally,4269,0
3-Doesnt Ship Everywhere,0,31378


In [9]:
structured_return_profile['structured_policy'] = 'No Policy'
structured_return_profile.loc[structured_return_profile['return_items_within_days']==7, 'structured_policy'] = '7 days'
structured_return_profile.loc[structured_return_profile['return_items_within_days']>=14, 'structured_policy'] = '>=14 days'
# structured_return_profile.loc[structured_return_profile['return_items_within_days']>14, 'structured_policy'] = '>14 days'
pd.crosstab(structured_return_profile['return_items_within_days'], structured_return_profile['structured_policy'])

structured_policy,7 days,>=14 days
return_items_within_days,,
7,5941,0
14,0,6016
21,0,3058
30,0,37287
45,0,256
60,0,98
90,0,144


In [10]:
TableOne(data = structured_return_profile, columns = ['ship_profile', 'seller_tier_new'],
         groupby = ['seller_country'],
          missing=False)

/opt/anaconda3/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),


Grouped by seller_country                                                       
                                                                                Overall EU seller/Omnibus affected Non-Omnibus affected seller
n                                                                                172837                     141459                       31378
ship_profile, n (%)    1-EU Origin/Ships Everywhere                       137190 (79.4)              137190 (97.0)                            
                       2-EU Origin Ship Country/Ships Locally                4269 (2.5)                 4269 (3.0)                            
                       3-Doesnt Ship Everywhere                            31378 (18.2)                                          31378 (100.0)
seller_tier_new, n (%) Closed Shop                                              1 (0.0)                    1 (0.0)                            
                       Empty Shop                                          30141 (17.4)               23885 (16.9)                 6256 (19.9)
                       Listed Shop                                         28319 (16.4)               21993 (15.5)                 6326 (20.2)
                       Medium Shop                                         32973 (19.1)               27597 (19.5)                 5376 (17.1)
                       Non Active Shop                                      10044 (5.8)                 7795 (5.5)                  2249 (7.2)
                       Power Shop                                            8684 (5.0)                 8204 (5.8)                   480 (1.5)
                       Small Shop                                          38428 (22.2)               30342 (21.4)                 8086 (25.8)
                       Top Shop                                            24247 (14.0)               21642 (15.3)                  2605 (8.3)

- <font color=blue> There are N=170,991 active traders (incoportaed sellers).
- <font color=blue> The Omnibus law affects 139,928 (81.8%) of all current active traders at Etsy, as they either resides in EU or ship to the EU.

In [11]:
TableOne(data = structured_return_profile[structured_return_profile['ship_profile']!='3-Doesnt Ship Everywhere'], 
         columns = ['seller_tier_new'], groupby = ['structured_policy'],
          missing=False)

/opt/anaconda3/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),


Grouped by structured_policy                                         
                                                            Overall       7 days     >=14 days     No Policy
n                                                            141459         5155         40527         95777
seller_tier_new, n (%) Closed Shop                          1 (0.0)                                  1 (0.0)
                       Empty Shop                      23885 (16.9)                             23885 (24.9)
                       Listed Shop                     21993 (15.5)    272 (5.3)   4379 (10.8)  17342 (18.1)
                       Medium Shop                     27597 (19.5)  1342 (26.0)  11165 (27.5)  15090 (15.8)
                       Non Active Shop                   7795 (5.5)                               7795 (8.1)
                       Power Shop                        8204 (5.8)  1188 (23.0)   5534 (13.7)    1482 (1.5)
                       Small Shop                      30342 (21.4)   561 (10.9)   7763 (19.2)  22018 (23.0)
                       Top Shop                        21642 (15.3)  1792 (34.8)  11686 (28.8)    8164 (8.5)

- <font color=blue> Among all affected traders (~140k), a total of 40,520 (28.6%) currently comply with Omnibus (offer >=14 days return policy)

In [12]:
# structured_return_profile['return_items_within_days'].describe()

In [13]:
# check2 = structured_return_profile[structured_return_profile['return_items_within_days'].isnull()]
# check2.shape

In [14]:
structured_return_profile['return_items_within_days'].value_counts(dropna=False)

NaN    120037
30      37287
14       6016
7        5941
21       3058
45        256
90        144
60         98
Name: return_items_within_days, dtype: Int64

### How many sellers are currently providing seller details on the shop site

In [15]:
query3 = """
with eu_countries as (select distinct country_id, 
                  iso_country_code,
                  name
            from `etsy-data-warehouse-prod.etsy_v2.countries`
            where name in('Belgium', 'Bulgaria', 'Czech Republic', 'Denmark', 'Cyprus', 'Latvia',
             'Lithuania', 'Luxembourg', 'Spain', 'France', 'Croatia', 'Italy', 'Poland', 'Portugal',
              'Romania', 'Slovenia', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Slovakia', 
              'Finland', 'Sweden', 'Germany', 'Estonia', 'Ireland', 'Greece')
),

-- active incorporated sellers who has a shipping profile (can ship to EU)
active_traders as (select distinct sb.shop_id,
sb.user_id,
sb.seller_tier_new,
sb.country_name, -- where the shop is based at
case when sb.country_name in (select name from eu_countries) then 'EU Seller' 
     else 'Other Seller' end as seller_country
from `etsy-data-warehouse-prod`.rollups.seller_basics sb
inner join `etsy-data-warehouse-prod.etsy_payments.shop_business_verification_attempts` kyb  -- filter on sellers who went through KYB to identify traders                                                
on sb.shop_id = kyb.shop_id    
inner join `etsy-data-warehouse-prod.etsy_shard.shipping_profile` s       -- filter out shops with no shipping profile? 
on sb.shop_id = s.shop_id 
where sb.user_state = 'active' -- active users only
and sb.shop_status = 'active' -- active shops only
and s.is_deleted = 0
),


-- get shipping profile shops (all active traders with active listings should be included in this table)
shipping_profile as (select distinct a.shop_id,
a.user_id,
min(case when a.country_name in (select name from eu_countries) and ships_everywhere = 0 then '2-EU Origin Ship Country/Ships Locally' 
      when ships_everywhere =1 then '1-EU Origin/Ships Everywhere' else '3-Doesnt Ship Everywhere' end) as ship_profile
from active_traders a
left join `etsy-data-warehouse-prod.etsy_shard.shipping_profile`  sp
on a.shop_id = sp.shop_id
left join `etsy-data-warehouse-prod.etsy_v2.countries` countries
on sp.origin_country_id = countries.country_id
where sp.is_deleted = 0 -- active shipping profile only
group by a.shop_id, a.user_id),

shops_provide_seller_detail as (select *
from (select shop_id,
details_id,
first_name,
last_name,	
address_line1,	
address_line2,
city,
state,
postal_code,
country,
vat_number,		
timestamp_seconds(update_date) as update_date,
rank() over (partition by shop_id order by timestamp_seconds(update_date) desc) as rnk,
email_address,
phone_number	
from etsy-data-warehouse-prod.etsy_shard.shop_seller_personal_details
--where shop_id in (31985053, 27603856)
)
where rnk = 1
)


-- join the tables
select a.shop_id,
a.seller_tier_new,
a.seller_country,
sp.ship_profile,
a.country_name,
b.details_id,
b.first_name,
b.last_name,	
b.address_line1,	
b.address_line2,
b.city,
b.state,
b.postal_code,
b.country,
b.vat_number,	
b.email_address,
b.phone_number	
from active_traders a
left join `etsy-data-warehouse-prod`.etsy_shard.shop_data s 
on a.shop_id = s.shop_id
left join shipping_profile sp
on a.shop_id = sp.shop_id
left join shops_provide_seller_detail b
on a.shop_id = b.shop_id
where ship_profile != '3-Doesnt Ship Everywhere' -- include countries that ship to EU only
"""

In [16]:
query_job = client.query(query3)
seller_details = pd.read_gbq(query=query3, project_id=project_id, 
#                                         progress_bar_type='tqdm'
                                    credentials=credentials)

In [17]:
seller_details.shape

(141459, 17)

In [18]:
print('There are N={} active traders who ships to EU and provides seller details on the shop page.'.format(seller_details[~seller_details['details_id'].isnull()]['shop_id'].nunique()))

There are N=13583 active traders who ships to EU and provides seller details on the shop page.


#### Label traders who provide seller details

In [25]:
seller_details.loc[~seller_details['details_id'].isnull(), 'provide_seller_details'] = 'Yes'
seller_details.loc[seller_details['details_id'].isnull(), 'provide_seller_details'] = 'No'
# pd.crosstab(seller_details['provide_seller_details'], 'counts')

#### Categorize seller tier to match with Sud's analysis

In [27]:
seller_details['seller_tier'] = 'Other Shops'
seller_details.loc[seller_details['seller_tier_new'] == 'Power Shop', 'seller_tier'] = 'Power Shops'
seller_details.loc[seller_details['seller_tier_new'] == 'Top Shop', 'seller_tier'] = 'Top Shops'
seller_details.loc[seller_details['seller_tier_new'] == 'Medium Shop', 'seller_tier'] = 'Medium Shops'
seller_details.loc[seller_details['seller_tier_new'] == 'Small Shop', 'seller_tier'] = 'Small Shops'
# pd.crosstab(seller_details['seller_tier_new'], seller_details['seller_tier'])

In [21]:
tab = pd.crosstab(seller_details['seller_tier'], seller_details['provide_seller_details'], 
            dropna=False, margins=True)
tab['percentage_no'] = round(100*tab['No']/tab['All'], 1)
tab['percentage_yes'] = round(100*tab['Yes']/tab['All'], 1)
tab = pd.DataFrame(tab, index=['Power Shops', 'Top Shops', 'Medium Shops', 'Small Shops', 'Other Shops', 'All'])
tab[['No', 'percentage_no', 'Yes', 'percentage_yes', 'All']]

provide_seller_details,No,percentage_no,Yes,percentage_yes,All
Power Shops,6443,78.5,1761,21.5,8204
Top Shops,18138,83.8,3504,16.2,21642
Medium Shops,24261,87.9,3336,12.1,27597
Small Shops,27812,91.7,2530,8.3,30342
Other Shops,51222,95.4,2452,4.6,53674
All,127876,90.4,13583,9.6,141459


<font color=blue> 1761 (21.5%) power shops and 3504 (16.2%) top shops currently provide seller detail information on their shop sites.